In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from timeit import default_timer

import paddle
from paddle.io import DataLoader, TensorDataset

from utilities3 import *
from catheter import FNO1d

paddle.seed(0)
np.random.seed(0)
paddle.set_device('gpu')

# Load data

In [ ]:
PATH = "/home/lunhao/Replication/AiScience/geofno-paddle/data/training/"
INPUT_X = PATH+"x_1d_structured_mesh.npy"
INPUT_Y = PATH+"y_1d_structured_mesh.npy"
INPUT_para = PATH+"data_info.npy"
OUTPUT = PATH+"density_1d_data.npy"
n_data = 3000
inputX_raw = np.load(INPUT_X)[:,0:n_data]
inputY_raw = np.load(INPUT_Y)[:,0:n_data]
inputPara_raw = np.load(INPUT_para)[:,0:n_data]
output_raw = np.load(OUTPUT)[:,0:n_data]



PATH = "/home/lunhao/Replication/AiScience/geofno-paddle/data/test/"
INPUT_X = PATH+"x_1d_structured_mesh.npy"
INPUT_Y = PATH+"y_1d_structured_mesh.npy"
INPUT_para = PATH+"data_info.npy"
OUTPUT = PATH+"density_1d_data.npy"
n_data = 300
inputX_test_raw = np.load(INPUT_X)[:,0:n_data]
inputY_test_raw = np.load(INPUT_Y)[:,0:n_data]
inputPara_test_raw = np.load(INPUT_para)[:,0:n_data]
output_test_raw = np.load(OUTPUT)[:,0:n_data]

N_s, L_x = 2001, 500


# Visualize sampled data

In [ ]:

plt.figure()
test_id = 0
plt.plot(inputX_raw[:,3*test_id], inputY_raw[:,3*test_id])
plt.plot(inputX_raw[:,3*test_id+1], inputY_raw[:,3*test_id+1])
plt.plot(inputX_raw[:,3*test_id+2], inputY_raw[:,3*test_id+2])
sample, uf, L_p, x1, x2, x3, h = inputPara_raw[:,3*test_id]

xx = np.linspace(-L_x, 0, N_s)
plt.plot(xx, output_raw[:,3*test_id], label="uf=5")
plt.plot(xx, output_raw[:,3*test_id+1], label="uf=10")
plt.plot(xx, output_raw[:,3*test_id+2], label="uf=15")
plt.plot(xx, (output_raw[:,3*test_id]+output_raw[:,3*test_id+1]+output_raw[:,3*test_id+2])/3, label="uf average")
plt.legend()

plt.figure()
plt.scatter(inputPara_raw[4,:], inputPara_raw[5,:])
plt.xlabel("x2")
plt.ylabel("x3")

plt.figure()
plt.scatter(inputPara_raw[4,:], inputPara_raw[2,:])
plt.plot(np.linspace(-150,0,100), np.linspace(60,60,100))
plt.xlabel("x2")
plt.ylabel("L_p")


# Process data

In [ ]:
# zero tests 

ntrain = 1000
ntest = 100

batch_size = 20
learning_rate = 0.001
epochs = 1001
step_size = 100
gamma = 0.5


modes = 64
width = 64
s = N_s

################################################################
# preprocess training data
################################################################
inputX = inputX_raw[:, 0::3]
inputY = inputY_raw[:, 0::3]
inputPara = inputPara_raw[:, 0::3]
output = (output_raw[:, 0::3] + output_raw[:, 1::3] + output_raw[:, 2::3])/ 3.0
intputX = paddle.to_tensor(inputX, dtype='float32').transpose([1,0])
intputY = paddle.to_tensor(inputY, dtype='float32').transpose([1,0])

tensor1 = paddle.randn([1000, 2001], dtype='float32')  # 使用randn生成随机数据作为示例
tensor2 = paddle.randn([1000, 2001], dtype='float32')  # 同样生成随机数据
tensor1 = intputX
tensor2 = intputY

input = paddle.stack([tensor1, tensor2], axis=2)
# input = paddle.stack([inputX, inputY], axis=2)
output = paddle.to_tensor(output, dtype='float32').transpose([1,0])
index = paddle.randperm(ntrain)
train_index = index[:ntrain]
x_train = paddle.index_select(input, train_index)
y_train = paddle.index_select(output, train_index)
x_train = x_train.reshape([ntrain, s, 2])
train_loader = DataLoader(TensorDataset([x_train, y_train]), batch_size=batch_size, shuffle=True)


################################################################
# preprocess test data
################################################################
inputX_test = inputX_test_raw[:, 0::3]
inputY_test = inputY_test_raw[:, 0::3]
inputPara_test = inputPara_test_raw[:, 0::3]
output_test = (output_test_raw[:, 0::3] + output_test_raw[:, 1::3] + output_test_raw[:, 2::3])/ 3.0
inputX_test = paddle.to_tensor(inputX_test, dtype='float32').transpose([1,0])
inputY_test = paddle.to_tensor(inputY_test, dtype='float32').transpose([1,0])
input_test = paddle.stack([inputX_test, inputY_test], axis=2)
output_test = paddle.to_tensor(output_test, dtype='float32').transpose([1,0])
x_test = input_test.reshape([ntest, s, 2])
y_test = output_test
test_loader = DataLoader(TensorDataset([x_test, y_test]), batch_size=batch_size, shuffle=True)

# Train Geo-FNO

Here we predicted the averaged bacteria distribution for three different flow rates.

In [ ]:
loss_data = np.zeros((3, epochs))

if __name__ == "__main__":
    ################################################################
    # training and evaluation
    ################################################################
    padding=100
    input_channel=2
    output_np=s

    model = FNO1d(modes, width, padding, input_channel=input_channel, output_np=output_np)
    model.set_state_dict(paddle.load("/home/lunhao/Replication/AiScience/geofno-paddle/model/GeoFno.pdparams"))
    print(count_params(model))

    scheduler = paddle.optimizer.lr.StepDecay(learning_rate=learning_rate, step_size=step_size, gamma=gamma)
    optimizer = paddle.optimizer.Adam(parameters=model.parameters(),learning_rate=scheduler, weight_decay=1e-4)

    myloss = LpLoss(size_average=False)
    
    for ep in range(epochs):
        model.train()
        t1 = default_timer()
        train_l2 = 0

        for batch_id, (x, y) in enumerate(train_loader()):

            out = paddle.exp(model(x))

            loss = myloss(out.view([batch_size, -1]), y.view([batch_size, -1]))
              
            loss.backward(retain_graph=True)

            optimizer.step()
            optimizer.clear_grad()
            train_l2 += loss.item()

        model.eval()
        test_l2 = 0.0
        for batch_id, (x, y) in enumerate(test_loader()):

            out = paddle.exp(model(x))

            # 计算损失与精度
            test_l2 += myloss(out.view([batch_size, -1]), y.view([batch_size, -1])).item()

        train_l2 /= ntrain
        test_l2 /= ntest

        t2 = default_timer()
        print(ep, t2 - t1, train_l2, test_l2)
        
        loss_data[:, ep] = ep, train_l2, test_l2
        # save nn
        if ep%step_size==0:
            paddle.save(model.state_dict(), '/home/lunhao/Replication/AiScience/geofno-paddle/model/catheter_plain_length_model_1d'+str(ep)+'.pdparams')
    

In [ ]:
plt.figure(figsize=(4,4))
plt.plot(loss_data[0, :], loss_data[1, :], "--", label="Training")
plt.plot(loss_data[0, :], loss_data[2, :], "--", label="Test")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.savefig("Loss.pdf")

# Verify the Geo-FNO model

In [ ]:
PATH = "/home/lunhao/Replication/AiScience/geofno-paddle/data/test/"
INPUT_X = PATH+"x_1d_structured_mesh.npy"
INPUT_Y = PATH+"y_1d_structured_mesh.npy"
INPUT_para = PATH+"data_info.npy"
OUTPUT = PATH+"density_1d_data.npy"

n_data = 300
inputX_raw = np.load(INPUT_X)[:,0:n_data]
inputY_raw = np.load(INPUT_Y)[:,0:n_data]
inputPara_raw = np.load(INPUT_para)[:,0:n_data]
output_raw = np.load(OUTPUT)[:,0:n_data]
inputPara_raw = np.load(INPUT_para)[:,0:n_data]

N_s, L_x = 2001, 500

model.set_state_dict(paddle.load("/home/lunhao/Replication/AiScience/geofno-paddle/model/catheter_plain_length_model_1d1000.pdparams"))

ntest = 100


modes = 64
width = 64
# nx ny
s = N_s

################################################################
# load data and data normalization
################################################################
inputX = inputX_raw[:, 0::3]
inputY = inputY_raw[:, 0::3]
inputPara = inputPara_raw[:, 0::3]
output = (output_raw[:, 0::3] + output_raw[:, 1::3] + output_raw[:, 2::3])/ 3.0


inputX = paddle.to_tensor(inputX, dtype=paddle.float32).transpose([1,0])
inputY = paddle.to_tensor(inputY, dtype=paddle.float32).transpose([1,0])
input = paddle.stack([inputX, inputY], axis=-1)
output = paddle.to_tensor(output, dtype=paddle.float32).transpose([1,0])

x_test = input.reshape([ntest, s, 2]) 
y_test = output 


for sample_id in [0,8]:
    sample, uf, L_p, x1, x2, x3, h = inputPara[:, sample_id]
    mesh = x_test[sample_id, :, :]
    
    y_test_pred = paddle.exp(model(x_test[sample_id:sample_id+1,:,:])).detach().cpu().numpy().flatten()
    print("rel. error is ", np.linalg.norm(y_test_pred - y_test[sample_id, :].numpy())/np.linalg.norm(y_test[sample_id, :]))
    xx = np.linspace(-L_x, 0 ,N_s)
    plt.figure(figsize=(5,4))
    
    
    
    plt.plot(mesh[:, 0].detach().cpu().numpy(), mesh[:, 1].detach().cpu().numpy(), color="C1", label="Channel geometry")
    plt.plot(mesh[:, 0].detach().cpu().numpy(), 100-mesh[:, 1].detach().cpu().numpy(), color="C1")
    
    plt.plot(xx, y_test[sample_id, :].detach().cpu().numpy(), "--o", color="red", markevery=len(xx)//10, label="Reference")
    plt.plot(xx, y_test_pred, "--*", color="C2", fillstyle='none', markevery=len(xx)//10, label="Predicted bacteria distribution")

    plt.xlabel(r"x")
    
    plt.legend()
    plt.tight_layout()
    plt.savefig("Validation."+str(sample_id)+".pdf")
    